In [2]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

# Läser in filer
movies_path = "../data/practise_movies.csv"
ratings_path = "../data/practise_ratings.csv"

df_movies = pd.read_csv(movies_path, usecols=["movieId", "title"])
df_ratings = pd.read_csv(ratings_path, usecols=["userId", "movieId", "rating"])

# Skapa en pivot-tabell för movieId och userId
movies_users = df_ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)

# Konvertera till sparse matrix
mat_movies_users = csr_matrix(movies_users.values)

# Skapa KNN-modellen
model_knn = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=20)
model_knn.fit(mat_movies_users)


def recommender(movie_name, data, model, n_recommendations=10):
    # Hitta index för den angivna filmen med fuzzy sökning
    query_index = process.extractOne(movie_name, df_movies['title'], score_cutoff=70)
    if query_index is None:
        print(f"Movie '{movie_name}' not found. Try another name.")
        return
    
    movie_idx = query_index[2]  # Indexet för filmen
    
    print(f"Movie Selected: {df_movies['title'][movie_idx]}, Index: {movie_idx}")
    print("Searching for recommendations...")
    
    # Hämta rekommendationer
    distances, indices = model.kneighbors(data[movie_idx], n_neighbors=n_recommendations+1)
    
    # Skriv ut rekommendationerna
    for i in indices.flatten():
        if i == movie_idx:
            continue
        print(df_movies['title'][i])

# Exempel på att använda funktionen
recommender("shrek", mat_movies_users, model_knn, 10)



Movie Selected: Shrek (2001), Index: 3194
Searching for recommendations...
Princess Mononoke (Mononoke-hime) (1997)
Million Dollar Hotel, The (2001)
Head Over Heels (2001)
Ideal Husband, An (1999)
America's Sweethearts (2001)
Loverboy (1989)
Others, The (2001)
Torn Curtain (1966)
Blue Crush (2002)
Written on the Wind (1956)



Med den här versionen av funktionen undviker man problem med indexering, förbättrar sökningen genom att inkludera en score_cutoff i process.extractOne för att säkerställa bättre matchningar, och korrekt filtrerar bort den valda filmen från resultatlistan av rekommendationer.